## 데이터 불러오기

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import dask.dataframe as dd
import vaex
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as spst
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
card_data = '../data/five_city_data/merged_gyeonggi_day.csv'
flowpop_data = '../data/five_city_data/flowpop.csv'
commercial_area_ndc_data = '../data/five_city_data/updated_commerical_area_ndc.csv'
card_sample = '../data/gm_card.csv'

In [3]:
# card_df = dd.read_csv(card_sample,encoding='utf-8')

In [4]:
card_df = dd.read_csv(card_data,encoding='utf-8')
flowpop_df = dd.read_csv(flowpop_data,encoding='utf-8')
comm_df = dd.read_csv(commercial_area_ndc_data,encoding='utf-8')

## 데이터 전처리

#### 카드 소비 데이터

In [5]:
card_df.head()

,ta_ymd,cty_rgn_no,admi_cty_no,card_tpbuz_cd,card_tpbuz_nm_1,card_tpbuz_nm_2,hour,sex,age,day,amt,cnt
0,20230101,41210,41210510,D05,소매/유통,선물/완구,3,M,5,7,57576,2
1,20230101,41210,41210510,D11,소매/유통,종합소매점,3,F,7,7,8523,2
2,20230101,41210,41210510,D11,소매/유통,종합소매점,7,F,7,7,8023,2
3,20230101,41210,41210510,F02,생활서비스,미용서비스,3,M,4,7,28532,2
4,20230101,41210,41210510,F02,생활서비스,미용서비스,4,F,7,7,74074,2


In [6]:
#card_df = card_df.compute()

In [7]:
#card_df.dtypes

In [8]:
flowpop_df.head()

,ETL_M,ADMI_CD,TIME_CD,M_10_CNT,F_10_CNT,M_20_CNT,F_20_CNT,M_30_CNT,F_30_CNT,M_40_CNT,F_40_CNT,M_50_CNT,F_50_CNT,M_60_CNT,F_60_CNT,M_70_CNT,F_70_CNT
0,2023-01,41111560,0,2858.84,2353.63,6248.39,4939.41,6684.28,4267.07,6304.54,5347.71,7495.55,5583.58,5411.89,4223.19,1086.22,1165.65
1,2023-01,41111560,1,3491.05,3021.34,7622.91,6183.19,7793.08,5365.17,7591.39,6765.04,9010.54,6842.45,6755.23,5363.00,1369.42,1499.24
2,2023-01,41111560,2,3433.43,3094.93,7369.46,5974.40,7510.31,5217.52,7282.29,6413.82,8717.53,6483.81,6631.32,5241.16,1303.01,1483.83
3,2023-01,41111560,3,3194.88,2955.40,7014.45,5704.24,7172.49,5138.34,6906.87,6334.15,8650.77,6395.10,6540.17,5316.28,1333.23,1501.99
4,2023-01,41111560,4,3003.63,2818.85,6716.24,5370.39,6980.61,4920.87,6908.53,6217.82,8911.40,6465.25,6940.02,5642.58,1519.33,1577.52


In [11]:
cnt_columns = flowpop_df.loc[:, 'M_10_CNT':'F_70_CNT'].columns

In [12]:
comm_df.head()

,행정동코드,기준분기,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분,상권변동지표구분명
0,41111560,1,99.9,46.4,HL,상권축소
1,41111566,1,83.5,43.2,LL,다이나믹
2,41111571,1,97.9,38.1,HL,상권축소
3,41111572,1,92.9,58.5,HH,정체
4,41111573,1,93.1,70.1,HH,정체


### 사용하지 않는 속성 제거

In [175]:
#flowpop_df.drop(['')],axis=1, inplace=True)

In [176]:
# 필요한 컬럼만 선택하여 병합
# merged_df = pd.merge(card_df[['ta_ymd', 'admi_cty_no', 'amt']], 
#                      flowpop_df[['TIME_CD', 'ADMI_CD', 'M_10_CNT', 'F_10_CNT']], 
#                      how='inner', 
#                      left_on=['ta_ymd', 'admi_cty_no'], 
#                      right_on=['TIME_CD', 'ADMI_CD'])

# 두 DataFrame을 날짜와 admi_cd를 기준으로 병합
#merged_df = dd.merge(card_df, flowpop_df, how='inner', left_on=['ta_ymd', 'admi_cty_no'], right_on=['ETL_YMD', 'ADMI_CD'])

# 중복 컬럼 제거
# merged_df.drop(columns=['TIME_CD', 'ADMI_CD'], inplace=True)

In [177]:
#merged_df.head()

## 카드 소비 데이터만 학습

### 범주형 데이터 변환

In [44]:
# enc_classes = {} 
# def encoding_label(x):   # x: 범주형 타입의 컬럼(Series)
#     le = LabelEncoder()
#     le.fit(x)
#     label = le.transform(x)
#     enc_classes[x.name] = le.classes_   # x.name: 컬럼명
#     return label

In [58]:
# card_df2 = card_df['card_tpbuz_cd'].apply(encoding_label)
# 오늘, 행정동에서, 어떤 업종에 방문한 연령대의 남성이 발생시킨 매출의 평균
grouped_df = card_df.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd", "sex", "age"]).agg({"amt": "mean", "cnt": "sum"}).reset_index()

grouped_df.head(5)

,ta_ymd,admi_cty_no,card_tpbuz_cd,sex,age,amt,cnt
0,20230101,41210510,D05,M,5,57576.0,2
1,20230101,41210510,D11,F,7,8273.0,4
2,20230101,41210510,F02,F,4,97650.0,2
3,20230101,41210510,F02,F,6,25789.0,2
4,20230101,41210510,F02,F,7,74074.0,2


In [59]:
grouped_df = grouped_df.drop(['ta_ymd'],axis=1)

In [60]:
train_df, test_df = train_test_split(grouped_df, test_size=0.2, random_state=42)

In [61]:
# train_size = int(len(card_df) * 0.8)
# train_df = card_df[:train_size]
# test_df = card_df[train_size:]

In [62]:
predictor = TabularPredictor(
    label='amt',  # 예측할 타겟 컬럼 설정
    eval_metric='mean_absolute_error'  # 평가 지표 설정
)

No path specified. Models will be saved in: "AutogluonModels\ag-20240711_033255"


In [ ]:
predictor.fit(train_data=train_df)

In [64]:
# 예측 수행
predictions = predictor.predict(test_df)

In [66]:
predictions.head()

1018321    3.377258e+05
2124564    2.413185e+04
2258131    1.342081e+06
2011751    2.614102e+05
1738195    7.445822e+04
Name: amt, dtype: float32

In [67]:
test_df.head()

,admi_cty_no,card_tpbuz_cd,sex,age,amt,cnt
1018321,41210590,S04,F,6,3.442950e+04,4
2124564,41210520,Q14,M,2,3.107500e+04,1
2258131,41210610,S04,M,7,3.930405e+06,66
2011751,41210550,F02,F,7,4.016005e+05,16
1738195,41210580,F03,M,4,6.081390e+06,2


In [68]:
predictor.leaderboard(test_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesMSE,-185856.722470,-185911.330726,mean_absolute_error,2.764904,0.123686,201.497280,2.764904,0.123686,201.497280,1,True,7
1,RandomForestMSE,-188120.379728,-187931.481231,mean_absolute_error,2.519405,0.124849,262.785172,2.519405,0.124849,262.785172,1,True,5
2,LightGBM,-190159.331894,-189084.193637,mean_absolute_error,3.645283,0.145639,19.794451,3.645283,0.145639,19.794451,1,True,4
3,LightGBMXT,-192731.062032,-192396.499009,mean_absolute_error,83.806180,3.146824,177.869431,83.806180,3.146824,177.869431,1,True,3
4,CatBoost,-216138.837812,-212478.484948,mean_absolute_error,0.667849,0.080087,1044.560808,0.667849,0.080087,1044.560808,1,True,6
5,KNeighborsUnif,-415725.153611,-446119.955703,mean_absolute_error,16.806362,1.236200,14.697853,16.806362,1.236200,14.697853,1,True,1
6,KNeighborsDist,-416299.692563,-446152.890681,mean_absolute_error,15.315404,0.645500,2.131970,15.315404,0.645500,2.131970,1,True,2


## 데이터 결합

### 월 별 데이터로 통합, 유동인구와 상권변화지표도 통합

In [13]:
# 해당일, 해당동의 해당 업종의 특정시간에 매출과 매출건수 총액
card_df2 = card_df.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd"]).agg({"amt": "sum", "cnt":"sum"}).reset_index()

In [14]:
card_df2.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt
0,20230101,41210510,D05,57576,2
1,20230101,41210510,D11,16546,4
2,20230101,41210510,F02,313388,16
3,20230101,41210510,Q01,69266,1
4,20230101,41210510,Q15,78222,6


In [15]:
card_df2['ta_ymd'] = card_df2['ta_ymd'].astype(str) 

In [16]:
card_df2['ta_ymd'] = card_df2['ta_ymd'].str.slice(0, 6)

In [17]:
card_df2.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt
0,202301,41210510,D05,57576,2
1,202301,41210510,D11,16546,4
2,202301,41210510,F02,313388,16
3,202301,41210510,Q01,69266,1
4,202301,41210510,Q15,78222,6


In [18]:
# 월별 행정구별, 업종별, amt 및 cnt 평균
grouped_df = card_df2.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd"]).agg({"amt": "sum", "cnt": "sum"}).reset_index()
grouped_df.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt
0,202301,41210510,D05,633562,28
1,202301,41210510,D11,171040,28
2,202301,41210510,F02,12060758,375
3,202301,41210510,Q01,1818710,9
4,202301,41210510,Q15,2167723,87


In [19]:
# join을 위한 데이터 통일
flowpop_df['ETL_M'] = flowpop_df['ETL_M'].str.replace('-', '')
# 총 유동인구 계산
cnt_columns = flowpop_df.loc[:, 'M_10_CNT':'F_70_CNT'].columns
flowpop_df['TOTAL_POPULATION'] = flowpop_df[cnt_columns].sum(axis=1)
# 해당 월에 해당 동에 해당 시간에 방문한 유동인구수 평균
flowpop_df_temp = flowpop_df.groupby(["ETL_M", "ADMI_CD","TIME_CD"]).agg({"TOTAL_POPULATION": "mean"}).reset_index()
# 해당 월에 해당 동에 방문한 유동인구 평균 총합
flow_g_df = flowpop_df_temp.groupby(["ETL_M", "ADMI_CD"]).agg({"TOTAL_POPULATION": "sum"}).reset_index()

In [20]:
flow_g_df.head()

,ETL_M,ADMI_CD,TOTAL_POPULATION
0,202301,41111560,2968697.82
1,202301,41111566,5438074.73
2,202301,41111571,2764829.73
3,202301,41111572,3664691.19
4,202301,41111573,5278122.27


In [21]:
# 데이터 결합
merged_df = dd.merge(grouped_df, flow_g_df, how='inner', left_on=['ta_ymd', 'admi_cty_no'], right_on=['ETL_M', 'ADMI_CD'])


In [22]:
merged_df.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt,ETL_M,ADMI_CD,TOTAL_POPULATION
0,202301,41210510,D05,633562,28,202301,41210510,13850.98
1,202301,41210510,D11,171040,28,202301,41210510,13850.98
2,202301,41210510,F02,12060758,375,202301,41210510,13850.98
3,202301,41210510,Q01,1818710,9,202301,41210510,13850.98
4,202301,41210510,Q15,2167723,87,202301,41210510,13850.98


In [23]:
# 기준분기 생성 함수
def get_quarter(month):
    if month in [1, 2, 3]:
        return 1
    elif month in [4, 5, 6]:
        return 2
    elif month in [7, 8, 9]:
        return 3
    elif month in [10, 11, 12]:
        return 4

# merged_df에서 기준분기 생성
merged_df['year'] = merged_df['ta_ymd'].astype(str).str[:4].astype(int)
merged_df['month'] = merged_df['ta_ymd'].astype(str).str[4:6].astype(int)
merged_df['기준분기'] = merged_df['month'].apply(get_quarter)

# comm_df에서 기준분기와 매칭될 수 있도록 연도를 추가
comm_df['year'] = 2023

C:\Users\DA\anaconda3\envs\hanati\lib\site-packages\dask\dataframe\core.py:4336: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('month', 'int64'))

  warnings.warn(meta_warning(meta))


In [24]:
merged_df.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt,ETL_M,ADMI_CD,TOTAL_POPULATION,year,month,기준분기
0,202301,41210510,D05,633562,28,202301,41210510,13850.98,2023,1,1
1,202301,41210510,D11,171040,28,202301,41210510,13850.98,2023,1,1
2,202301,41210510,F02,12060758,375,202301,41210510,13850.98,2023,1,1
3,202301,41210510,Q01,1818710,9,202301,41210510,13850.98,2023,1,1
4,202301,41210510,Q15,2167723,87,202301,41210510,13850.98,2023,1,1


In [25]:
comm_df.head()

,행정동코드,기준분기,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분,상권변동지표구분명,year
0,41111560,1,99.9,46.4,HL,상권축소,2023
1,41111566,1,83.5,43.2,LL,다이나믹,2023
2,41111571,1,97.9,38.1,HL,상권축소,2023
3,41111572,1,92.9,58.5,HH,정체,2023
4,41111573,1,93.1,70.1,HH,정체,2023


In [26]:
# comm_df와 merged_df 병합
merged_result = dd.merge(merged_df, comm_df, left_on=['admi_cty_no', '기준분기', 'year'], right_on=['행정동코드', '기준분기', 'year'], how='left')

# 병합 결과에서 필요없는 열 제거 (예: '행정동코드')
merged_result = merged_result.drop(columns=['행정동코드'])

# 병합 결과 출력
merged_result.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt,ETL_M,ADMI_CD,TOTAL_POPULATION,year,month,기준분기,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분,상권변동지표구분명
0,202301,41210510,D05,633562,28,202301,41210510,13850.98,2023,1,1,188.5,358.0,HH,정체
1,202301,41210510,D11,171040,28,202301,41210510,13850.98,2023,1,1,188.5,358.0,HH,정체
2,202301,41210510,F02,12060758,375,202301,41210510,13850.98,2023,1,1,188.5,358.0,HH,정체
3,202301,41210510,Q01,1818710,9,202301,41210510,13850.98,2023,1,1,188.5,358.0,HH,정체
4,202301,41210510,Q15,2167723,87,202301,41210510,13850.98,2023,1,1,188.5,358.0,HH,정체


In [63]:
len(comm_df[comm_df['행정동코드'] == 41210510])

3

In [27]:
# 해당 행정동의 상권 지표를 결합
merged_df2 = dd.merge(merged_df, comm_df, how='inner', left_on=['admi_cty_no'], right_on=['행정동코드'])

In [28]:
#merged_result.isna().sum()

In [29]:
merged_df2.columns

Index(['ta_ymd', 'admi_cty_no', 'card_tpbuz_cd', 'amt', 'cnt', 'ETL_M',
       'ADMI_CD', 'TOTAL_POPULATION', 'year_x', 'month', '기준분기_x', '행정동코드',
       '기준분기_y', '운영점포평균영업기간', '폐업점포평균영업기간', '상권변동지표구분', '상권변동지표구분명',
       'year_y'],
      dtype='object')

In [30]:
# 필요없는 속성 제거
# columns_to_drop = ['ta_ymd','ETL_M', 'ADMI_CD', '행정시명', '행정동명',
#                    '운영점포평균영업기간', '폐업점포평균영업기간', '상권변동지표구분명', 'year','month','기준분기']

# columns_to_drop = ['ta_ymd','ETL_M', 'ADMI_CD', '운영점포평균영업기간', '폐업점포평균영업기간', '상권변동지표구분명', 'year','month','기준분기']
# merged_df2 = merged_df2.drop(columns=columns_to_drop, meta={'ta_ymd': 'int64', 'ETL_M': 'int64', 'ADMI_CD': 'int64', 
#                                                                  '운영점포평균영업기간': 'float64', '폐업점포평균영업기간': 'float64', 
#                                                                  '상권변동지표구분명': 'object', 'year': 'int64', 'month': 'int64', '기준분기': 'int64'})


columns_to_drop = ['ta_ymd','ETL_M', 'ADMI_CD', 'year_x','기준분기_x', '행정동코드','기준분기_y',
                   '상권변동지표구분명', 'year_y','month']
merged_df2 = merged_df2.drop(columns=columns_to_drop)

In [31]:
merged_df2.head()

,admi_cty_no,card_tpbuz_cd,amt,cnt,TOTAL_POPULATION,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분
0,41210510,D05,633562,28,13850.98,188.5,358.0,HH
1,41210510,D05,633562,28,13850.98,188.5,358.0,HH
2,41210510,D05,633562,28,13850.98,188.5,358.0,HH
3,41210510,D11,171040,28,13850.98,188.5,358.0,HH
4,41210510,D11,171040,28,13850.98,188.5,358.0,HH


In [32]:
# 505027 rows
len(merged_df2)

403080

## One hot encoding, train-test split

In [34]:
from multiprocessing import Pool, cpu_count
cpu_count()

12

In [33]:
merged_df2 = merged_df2.compute()
merged_df2_encoded = pd.get_dummies(merged_df2, columns=['card_tpbuz_cd', '상권변동지표구분'])

# 변환된 DataFrame 확인
print(merged_df2_encoded.head())

# train/test 데이터 나누기 (80-20 비율)
train_df, test_df = train_test_split(merged_df2_encoded, test_size=0.2, random_state=42)

   admi_cty_no     amt  cnt  TOTAL_POPULATION  운영점포평균영업기간  폐업점포평균영업기간  \
0     41210510  633562   28          13850.98       188.5       358.0   
1     41210510  633562   28          13850.98       188.5       358.0   
2     41210510  633562   28          13850.98       188.5       358.0   
3     41210510  171040   28          13850.98       188.5       358.0   
4     41210510  171040   28          13850.98       188.5       358.0   

   card_tpbuz_cd_D01  card_tpbuz_cd_D02  card_tpbuz_cd_D03  card_tpbuz_cd_D04  \
0              False              False              False              False   
1              False              False              False              False   
2              False              False              False              False   
3              False              False              False              False   
4              False              False              False              False   

   ...  card_tpbuz_cd_U04  card_tpbuz_cd_Y01  card_tpbuz_cd_Y02  \
0  ... 

## 모델 학습1

In [35]:
# Autogluon TabularPredictor 사용
predictor = TabularPredictor(
    label='amt',  # 예측할 타겟 컬럼 설정
    eval_metric='mean_absolute_error'  # 평가 지표 설정 
)

# 모델 훈련
predictor.fit(
    train_data=train_df,
    presets='good_quality_faster_inference_only_refit',  # 가벼운 모델 설정
    time_limit=3000  # 학습 시간 2시간으로 제한 (7200초)
)

# 예측 수행
predictions = predictor.predict(test_df)

# 예측 결과 및 실제 값 비교
print("Predictions:\n", predictions)
print("Actual:\n", test_df['amt'])

No path specified. Models will be saved in: "AutogluonModels\ag-20240712_120603"
Preset alias specified: 'good_quality_faster_inference_only_refit' maps to 'good_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.18
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
Memory Avail:       16.52 GB / 31.69 GB (52.1%)
Disk Space Avail:   573.47 GB / 930.86 GB (61.6%)
Presets specified: ['good_quality_faster_inference_only_refit']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data

(_ray_fit pid=28068) [1000]	valid_set's l1: 9.03205e+07
(_ray_fit pid=28068) [2000]	valid_set's l1: 7.56564e+07 [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=26136) [3000]	valid_set's l1: 7.12336e+07 [repeated 8x across cluster]
(_ray_fit pid=26136) [4000]	valid_set's l1: 6.63103e+07 [repeated 8x across cluster]
(_ray_fit pid=26136) [5000]	valid_set's l1: 6.26884e+07 [repeated 8x across cluster]
(_ray_fit pid=28068) [6000]	valid_set's l1: 5.72368e+07 [repeated 8x across cluster]
(_ray_fit pid=28068) [7000]	valid_set's l1: 5.51711e+07 [repeated 8x across cluster]
(_ray_fit pid=21952) [7000]	valid_set's l1: 5.50386e+07 [repeated 7x across cluster]
(_ray_fit pid=26136) [8000]	valid_set's l1: 5.63539e+07
(_ray_fit pid=28068) [8000]	valid_set's l1: 5.35417e+07
(_ray_fit pid=28068) [9000]	valid_set's l1

(_dystack pid=7772) 	-52568906.9247	 = Validation score   (-mean_absolute_error)
(_dystack pid=7772) 	152.41s	 = Training   runtime
(_dystack pid=7772) 	1084.81s	 = Validation runtime
(_dystack pid=7772) Fitting model: LightGBM_BAG_L1 ... Training model for up to 198.89s of the 444.95s of remaining time.
(_dystack pid=7772) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.56%)


(_ray_fit pid=25664) [1000]	valid_set's l1: 6.56461e+07
(_ray_fit pid=21300) [1000]	valid_set's l1: 6.13972e+07
(_ray_fit pid=25664) [2000]	valid_set's l1: 5.50635e+07 [repeated 7x across cluster]
(_ray_fit pid=6052) [3000]	valid_set's l1: 4.46114e+07 [repeated 8x across cluster]
(_ray_fit pid=6052) [4000]	valid_set's l1: 4.18803e+07 [repeated 8x across cluster]
(_ray_fit pid=19200) [5000]	valid_set's l1: 4.00937e+07 [repeated 8x across cluster]
(_ray_fit pid=19200) [6000]	valid_set's l1: 3.89744e+07 [repeated 8x across cluster]
(_ray_fit pid=19200) [7000]	valid_set's l1: 3.79277e+07 [repeated 8x across cluster]
(_ray_fit pid=19200) [8000]	valid_set's l1: 3.69405e+07 [repeated 8x across cluster]
(_ray_fit pid=24632) [9000]	valid_set's l1: 3.4709e+07 [repeated 8x across cluster]
(_ray_fit pid=19200) [10000]	valid_set's l1: 3.54324e+07 [repeated 8x across cluster]


(_dystack pid=7772) 	-36329424.4495	 = Validation score   (-mean_absolute_error)
(_dystack pid=7772) 	130.42s	 = Training   runtime
(_dystack pid=7772) 	896.15s	 = Validation runtime
(_dystack pid=7772) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 199.19s of remaining time.
(_dystack pid=7772) 	Ensemble Weights: {'LightGBM_BAG_L1': 0.96, 'LightGBMXT_BAG_L1': 0.04}
(_dystack pid=7772) 	-36291918.5971	 = Validation score   (-mean_absolute_error)
(_dystack pid=7772) 	0.3s	 = Training   runtime
(_dystack pid=7772) 	0.01s	 = Validation runtime
(_dystack pid=7772) Fitting 9 L2 models ...
(_dystack pid=7772) Fitting model: LightGBMXT_BAG_L2 ... Training model for up to 198.86s of the 198.84s of remaining time.
(_dystack pid=7772) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.79%)


(_ray_fit pid=30396) [1000]	valid_set's l1: 5.70715e+07 [repeated 8x across cluster]
(_ray_fit pid=21884) [2000]	valid_set's l1: 5.322e+07 [repeated 8x across cluster]
(_ray_fit pid=21884) [3000]	valid_set's l1: 5.0733e+07 [repeated 8x across cluster]
(_ray_fit pid=21884) [4000]	valid_set's l1: 4.92898e+07 [repeated 8x across cluster]
(_ray_fit pid=21884) [5000]	valid_set's l1: 4.82508e+07 [repeated 8x across cluster]
(_ray_fit pid=21884) [6000]	valid_set's l1: 4.73186e+07 [repeated 8x across cluster]
(_ray_fit pid=21884) [7000]	valid_set's l1: 4.66617e+07 [repeated 8x across cluster]
(_ray_fit pid=26184) [7000]	valid_set's l1: 4.27735e+07 [repeated 7x across cluster]
(_ray_fit pid=21884) [8000]	valid_set's l1: 4.60792e+07
(_ray_fit pid=30396) [8000]	valid_set's l1: 4.37819e+07
(_ray_fit pid=26184) [8000]	valid_set's l1: 4.21723e+07 [repeated 6x across cluster]
(_ray_fit pid=21884) [9000]	valid_set's l1: 4.56386e+07
(_ray_fit pid=30396) [9000]	valid_set's l1: 4.32475e+07
(_ray_fit pid=

(_dystack pid=7772) 	-42775660.4627	 = Validation score   (-mean_absolute_error)
(_dystack pid=7772) 	156.99s	 = Training   runtime
(_dystack pid=7772) 	922.37s	 = Validation runtime
(_dystack pid=7772) Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -72.69s of remaining time.
(_dystack pid=7772) 	Ensemble Weights: {'LightGBM_BAG_L1': 0.833, 'LightGBMXT_BAG_L2': 0.167}
(_dystack pid=7772) 	-35905644.3224	 = Validation score   (-mean_absolute_error)
(_dystack pid=7772) 	0.39s	 = Training   runtime
(_dystack pid=7772) 	0.0s	 = Validation runtime
(_dystack pid=7772) AutoGluon training complete, total runtime = 812.69s ... Best model: WeightedEnsemble_L3 | Estimated inference throughput: 12.3 rows/s (35830 batch size)
(_dystack pid=7772) Automatically performing refit_full as a post-fit operation (due to `.fit(..., refit_full=True)`
(_dystack pid=7772) Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
(_dystac

Predictions:
 388390    2.953763e+08
93442     1.892789e+07
396598    1.648337e+08
166263    1.853853e+08
88675     3.070225e+07
              ...     
155526    2.062783e+08
168829    1.451311e+09
153934    3.056364e+08
259348    2.506232e+07
310736    6.549724e+08
Name: amt, Length: 80616, dtype: float32
Actual:
 388390     365433405
93442       15489679
396598     193816450
166263     204928779
88675       23182820
             ...    
155526     183013621
168829    1543539840
153934     362683500
259348      29006955
310736     613019243
Name: amt, Length: 80616, dtype: int64


In [36]:
y_pred = predictor.predict(test_df.drop(columns=['amt']))
y_pred.head()

388390    295376288.0
93442      18927890.0
396598    164833744.0
166263    185385312.0
88675      30702254.0
Name: amt, dtype: float32

In [37]:
predictor.evaluate(test_df, silent=True)

{'mean_absolute_error': -24131858.03846056,
 'root_mean_squared_error': -130607424.8679933,
 'mean_squared_error': -1.7058299430648514e+16,
 'r2': 0.994546865502877,
 'pearsonr': 0.9973066891807696,
 'median_absolute_error': -7124283.0}

In [38]:
predictor.leaderboard(test_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3_FULL,-2.413186e+07,NaN,mean_absolute_error,32.211538,NaN,487.052603,0.002990,NaN,1.135359,3,True,20
1,LightGBM_BAG_L2_FULL,-2.413257e+07,NaN,mean_absolute_error,32.208548,NaN,485.917244,9.559220,NaN,31.981132,2,True,18
2,LightGBM_BAG_L1_FULL,-3.090461e+07,NaN,mean_absolute_error,9.630037,NaN,27.587677,9.630037,NaN,27.587677,1,True,12
3,WeightedEnsemble_L2_FULL,-3.094170e+07,NaN,mean_absolute_error,21.924952,NaN,59.605852,0.034114,NaN,0.785178,2,True,16
4,LightGBMXT_BAG_L2_FULL,-3.496090e+07,NaN,mean_absolute_error,33.678393,NaN,488.483329,11.029065,NaN,34.547216,2,True,17
5,CatBoost_BAG_L2_FULL,-3.555428e+07,NaN,mean_absolute_error,22.682241,NaN,463.965286,0.032913,NaN,10.029173,2,True,19
6,LightGBMXT_BAG_L1_FULL,-4.861183e+07,NaN,mean_absolute_error,12.260801,NaN,31.232996,12.260801,NaN,31.232996,1,True,11
7,CatBoost_BAG_L1_FULL,-5.551754e+07,NaN,mean_absolute_error,0.102832,NaN,79.681405,0.102832,NaN,79.681405,1,True,14
8,RandomForestMSE_BAG_L1,-7.438668e+07,-7.658554e+07,mean_absolute_error,0.423140,13.610413,219.873307,0.423140,13.610413,219.873307,1,True,3
9,RandomForestMSE_BAG_L1_FULL,-7.438668e+07,NaN,mean_absolute_error,0.438187,13.610413,219.873307,0.438187,13.610413,219.873307,1,True,13
